# 13. Redes Neuronales recurrentes
[**Python Deep Learning** Introducción práctica con Keras y TensorFlow 2. Jordi Torres. Editorial Marcombo ISBN: 9788426728289 ](https://www.marcombo.com/python-deep-learning-9788426728289/)

### Importar TensorFlow 2.0  y otras librerias

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

import numpy as np
import os
import time

TensorFlow 2.x selected.


### Descarga del conjunto de datos

In [0]:
path_to_fileDL = tf.keras.utils.get_file('DL-Introduccion-practica-con-Keras-1aParte.txt', 'https://raw.githubusercontent.com/jorditorresBCN/Deep-Learning-Introduccion-practica-con-Keras/master/DeepLearning-Introduccion-practica-con-Keras-PRIMERA-PARTE.txt')  

#path_to_fileDL = tf.keras.utils.get_file('Shakespear.txt', 'https://cs.stanford.edu/people/karpathy/char-rnn/shakespear.txt')

text = open(path_to_fileDL, 'rb').read().decode(encoding='utf-8')
print('Longitud del texto:        {} carácteres'.format(len(text)))

vocab = sorted(set(text))

print ('El texto está compuesto de estos {} carácteres:'.format(len(vocab)))
print (vocab)

204800/203286 [==============================] - 0s 0us/step
Longitud del texto:        203251 carácteres
El texto está compuesto de estos 92 carácteres:
['\n', '\r', ' ', '!', '"', '#', '%', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xad', 'ÿ', 'Š', '‡', '…']


In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

for char,_ in zip(char2idx, range(len(vocab))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '#' :   5,
  '%' :   6,
  "'" :   7,
  '(' :   8,
  ')' :   9,
  '*' :  10,
  '+' :  11,
  ',' :  12,
  '-' :  13,
  '.' :  14,
  '/' :  15,
  '0' :  16,
  '1' :  17,
  '2' :  18,
  '3' :  19,
  '4' :  20,
  '5' :  21,
  '6' :  22,
  '7' :  23,
  '8' :  24,
  '9' :  25,
  ':' :  26,
  ';' :  27,
  '<' :  28,
  '=' :  29,
  '>' :  30,
  '?' :  31,
  '@' :  32,
  'A' :  33,
  'B' :  34,
  'C' :  35,
  'D' :  36,
  'E' :  37,
  'F' :  38,
  'G' :  39,
  'H' :  40,
  'I' :  41,
  'J' :  42,
  'K' :  43,
  'L' :  44,
  'M' :  45,
  'N' :  46,
  'O' :  47,
  'P' :  48,
  'Q' :  49,
  'R' :  50,
  'S' :  51,
  'T' :  52,
  'U' :  53,
  'V' :  54,
  'W' :  55,
  'X' :  56,
  'Y' :  57,
  '[' :  58,
  ']' :  59,
  '_' :  60,
  'a' :  61,
  'b' :  62,
  'c' :  63,
  'd' :  64,
  'e' :  65,
  'f' :  66,
  'g' :  67,
  'h' :  68,
  'i' :  69,
  'j' :  70,
  'k' :  71,
  'l' :  72,
  'm' :  73,
  'n' :  74,
  'o' :  75,
  'p' :  76,

In [0]:
text_as_int = np.array([char2idx[c] for c in text])

In [0]:
print ('texto: {}'.format(repr(text[:50])))
print ('{}'.format(repr(text_as_int[:50])))

texto: 'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fun'
array([48, 78, 75, 72, 75, 67, 75,  1,  0, 37, 74,  2, 17, 25, 21, 19, 12,
        2, 41, 79, 61, 61, 63,  2, 33, 79, 69, 73, 75, 82,  2, 76, 81, 62,
       72, 69, 63, 75,  2, 51, 65, 67, 81, 74, 64, 61,  2, 38, 81, 74])


### Preparar los datos para entrenar la RNN


In [0]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

seq_length = 100
 
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)



In [0]:
for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion '
'(o el decimotercero segun otras fuentes, este es un tema de debate). En Segunda Fundacion aparece por'
' primera vez Arkady Darell, uno de los principales personajes de la parte final de la saga. En su pri'
'mera escena, Arkady, que tiene 14 anos, esta haciendo sus tareas escolares. En concreto, una redaccio'
'n que lleva por titulo ?El Futuro del Plan Sheldon?. Para hacer la redaccion, Arkady esta utilizando '
'un ?transcriptor?,un dispositivo que convierte su voz en palabras escritas. Este tipo de dispositivo,'
' que para Isaac Asimov era ciencia ficcion en 1953, lo tenemos al alcance de la mano en la mayoria de'
' nuestros smartphones, y el Deep Learning es uno de los responsables de que ya tengamos este tipo de '
'aplicaciones, siendo la tecnologia otro de ellos.En la actualidad disponemos de GPUs (Graphics Proces'
'sor Units), que solo cuestan alrededor de 100 euros, que esta

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion'
Target data: 'rologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion '


In [0]:
print (dataset)

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>


In [0]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print (dataset)

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


### Construcción del modelo

In [0]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23552     
_________________________________________________________________
unified_lstm (UnifiedLSTM)   (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 92)            94300     
Total params: 5,364,828
Trainable params: 5,364,828
Non-trainable params: 0
_________________________________________________________________


In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")


Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)


In [0]:
for input_example_batch, target_example_batch in dataset.take(1):  
    example_batch_predictions = model(input_example_batch)
    print("Prediction: ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Prediction:  (64, 100, 92) # (batch_size, sequence_length, vocab_size)


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices_characters = tf.squeeze(sampled_indices,axis=-1).numpy()

In [0]:
sampled_indices_characters

array([ 1, 89, 85, 20,  6, 73, 54, 40,  7, 57,  3, 67, 81, 47, 19, 14, 81,
       35, 78, 37, 90, 34, 50, 46, 35,  7, 73,  4, 59, 75,  6, 48, 28, 82,
       14, 78,  2, 68, 72, 23, 87, 66, 70, 19, 78, 38, 48, 65,  6, 36, 87,
       66, 69, 76, 57,  2, 89, 54, 26, 57, 37, 12,  1, 47,  0, 51, 52, 18,
       48, 87,  9, 48, 55, 67, 79, 22, 54, 53, 49,  2, 17, 76, 40, 32, 60,
       59, 22, 32, 82, 47, 35, 59, 66, 86,  8, 39, 60, 61, 68, 40])

Entrenar el modelo

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
model.compile(optimizer='adam', loss=loss)

Configurar el *checkpoints*

In [0]:
 # directorio
checkpoint_dir = './training_checkpoints'
# nombre fichero
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

*Training*

In [0]:
EPOCHS=50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
31/31 [==============================] - 3s 107ms/step - loss: 3.2626
Epoch 2/50
31/31 [==============================] - 3s 82ms/step - loss: 2.7938
Epoch 3/50
31/31 [==============================] - 3s 81ms/step - loss: 2.4345
Epoch 4/50
31/31 [==============================] - 3s 83ms/step - loss: 2.2087
Epoch 5/50
31/31 [==============================] - 3s 83ms/step - loss: 2.0815
Epoch 6/50
31/31 [==============================] - 3s 83ms/step - loss: 1.9603
Epoch 7/50
31/31 [==============================] - 3s 84ms/step - loss: 1.8362
Epoch 8/50
31/31 [==============================] - 3s 86ms/step - loss: 1.7192
Epoch 9/50
31/31 [==============================] - 3s 88ms/step - loss: 1.6094
Epoch 10/50
31/31 [==============================] - 3s 86ms/step - loss: 1.5072
Epoch 11/50
31/31 [==============================] - 3s 87ms/step - loss: 1.4165
Epoch 12/50
31/31 [==============================] - 3s 88ms/step - loss: 1.3356
Epoch 13/50
31/31 [=================

### Generación de texto

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_50'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
def generate_text(model, start_string):

  num_generate = 500
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []


  temperature = 0.5

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      
      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()


      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"Alcohol "))

Alcohol 


EBUNK9‡+[I**JJJxJK‡@'D]+k'J[‡!9‡DBX*X*WOIB­xRM‡]#WJ['IW"@‡Wx@I%WGNYI%‡#xY[‡[%B[DIK‡]AQBJ/DB!W"YYR‡?'IWQ%*YJII­H+B3Dx‡[A‡NQY*­5JQ+XIJwJWWJ"N7D]YBCR‡YI‡%NHDK
U@[x3I­<L!wJ'N9INzw@HB"9J%>L…#JQJB#R<[L
G%IhBD6JK'#]UHIVIY[BTJ<HPN]Y!'ÿ[!‡%*‡]RWJ%Y[D[R%H!JYUIH*I[[4kWQ'B]‡;'Y*[D9%‡WJ<[J*OGNB9
JBNwJR'WJBKI#T9Jw?JYI3#[TQ@UJ*UGIQ!@V""V9‡6"JBG'3K'Yw‡TJ%IAzIQJ+''‡]DA9JJBHJ[F‡FxB"<I3H9AhW
x[JJHY‡BIJB#W<J9]‡2‡QIX"JJ9GBQx;I‡xH]UD[GJJJ>"j%x[zHH<JD[w"*HH+I9Y[DÿD[INBR[DUGY+J#DwH!O[‡"YX'KWJK"'IY‡IHJJJ;9>D4‡YV9JKW


In [0]:
print(generate_text(model, start_string=u"modelo"))

modelo se usa para implementar modelos de redes muy grandes.  Para nuestro ejemplo simple, vemos que indica que se requiere mucha expericencambiendo su funcionamos el proceso de aprendizaje de una red neuronal. Ademas veremos algunos de los parametros W y b de tal manera que se minimice la funcion de loss que usaremos para evaluar el grado de error es un espacio de dos cientificos y cientificas de datos y que puedan solurio en la comunidad de programadores ya que permite ample mescampo. Ahora, es el t


In [0]:
print(generate_text(model, start_string=u"activacion"))

activacion si la primera visto y es usaramos una de las predicciones por computador ya se conocian en 1989; tambien los algoritmos fundamentales de Deep Learning puede hacerlo a traves de la como de la UPC model prodecir esta siguiente paralelas Deep Learning ello tenemos otra metrica llamada Sensitivity (o recall) que nos indica como de bien el modelo evita el BSC para referirse a las diferentes actualizaciones de su supercomputador Marenostrum que tanta la provecho de las nuevas se pasan en glabal de da
